# Test Cache in Jupyter Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import logging

import joblib

import helpers.cache as hcac
import helpers.dbg as dbg
import helpers.printing as prnt
import helpers.s3 as hs3

prnt.config_notebook()

# dbg.init_logger(verbosity=logging.DEBUG)
dbg.init_logger(verbosity=logging.INFO)
# dbg.test_logger()
_LOG = logging.getLogger(__name__)

# Define computation function

In [ ]:
def func(a, b):
    # hello
    # assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

func(*inputs)

In [ ]:
!ls hello/joblib/__main__*/f/

In [ ]:
!pip install https://github.com/aabadie/joblib-s3.git

In [ ]:
#!git clone git://github.com/aabadie/joblib-s3.git
# !(cd joblib-s3 && pip install -r requirements.txt .)

In [ ]:
# import joblibs3

# joblibs3.register_s3fs_store_backend()

# # dict(compress=False, bucket=None, anon=False,
#                                #key=None, secret=None, token=None, use_ssl=True)
# dict2 = {
#     "bucket": "alphamatic-data",
#     "key": dict_["aws_access_key_id"],
#     "secret": dict_["aws_secret_access_key"],
# }
# mem = joblib.Memory('joblib_cache', backend='s3', verbose=100, compress=True,
#                  backend_options=dict2)

In [ ]:
# hjoblib.register_s3fs_store_backend()

s3fs = hs3.get_s3fs("am")

dict2 = {
    "bucket": "alphamatic-data",
    # "key": dict_["aws_access_key_id"],
    # "secret": dict_["aws_secret_access_key"],
    "s3fs": s3fs,
}

mem = joblib.Memory(
    "joblib_cache",
    backend="s3",
    verbose=100,
    compress=True,
    backend_options=dict2,
)

In [ ]:
# hjoblib.register_s3fs_store_backend()

s3fs = hs3.get_s3fs("am")

dict2 = {
    "bucket": "alphamatic-data",
    # "key": dict_["aws_access_key_id"],
    # "secret": dict_["aws_secret_access_key"],
    "s3fs": s3fs,
}
path = "/tmp/cache.unit_test/root.98e1cf5b88c3.app.TestCachingOnS3.test_with_caching1"


s3fs.ls(path)

# mem = joblib.Memory(path, backend='s3', verbose=100, compress=True, backend_options=dict2)

In [ ]:
print(dict_)

In [ ]:
# dict_["bucket"] = "alphamatic-data/tmp"

print(dict_)

In [ ]:
def dec(func=None, val=5):
    if func is not None:
        return

In [ ]:
dict_ = hs3.get_aws_credentials("am")
print(dict_)
# s3fs = hs3.get_s3fs("am")
# s3fs.ls("s3://alphamatic-data/tmp")

In [ ]:
s3fs.clear_instance_cache()

In [ ]:
# import joblib

# cachedir = "./hello"
# memory = joblib.Memory(cachedir, verbose=0)


@mem.cache()
def f(x):
    # hello
    print("Running f(%s)" % x)
    return x


f(1)

In [ ]:
hcac.cache(set_verbose_mode=True)


def hello():
    return "hello"


hello()

## Memory cache

In [ ]:
!ls /app/tmp.cache.disk/joblib/

In [ ]:
!ls /mnt/tmpfs/tmp.cache.mem/joblib/lib

In [ ]:
memory_cached_func = hcac._Cached(func, use_mem_cache=True, use_disk_cache=False)

print(memory_cached_func.get_function_cache_info())

# cache_type = None
# memory_cached_func.clear_function_cache(cache_type)

dbg.dassert_eq(memory_cached_func(*inputs), exp_output)
dbg.dassert_eq(memory_cached_func.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(memory_cached_func(*inputs), exp_output)
dbg.dassert_eq(memory_cached_func.get_last_cache_accessed(), "mem")

print("memory caching checks passed")

In [ ]:
def computation_function(a, b):
    # hello
    # assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

## Disk cache

In [ ]:
disk_cached_computation = hcac._Cached(
    computation_function, use_mem_cache=False, use_disk_cache=True
)

disk_cached_computation.clear_function_cache("disk")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

print("disk caching checks passed")

## Full cache

In [ ]:
fully_cached_computation = hcac._Cached(
    computation_function, use_mem_cache=True, use_disk_cache=True
)

fully_cached_computation.clear_function_cache()

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("Clear mem cache")
fully_cached_computation.clear_function_cache("mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "disk")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("full caching checks passed")

In [ ]:
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

In [ ]:
# This should fail all the times, because we clear the memory cache.
fully_cached_computation.clear_function_cache("mem")
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")